In [ ]:
!pip install  git+https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git
!pip install  -q git+https://github.com/MarcusLoppe/meshgpt-pytorch.git

In [33]:
import torch
import trimesh
import numpy as np
import os
import csv
from collections import OrderedDict

from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
) 

def get_mesh(file_path): 
    mesh = trimesh.load(file_path, force='mesh')
     
    vertices = mesh.vertices.tolist()
    faces = mesh.faces.tolist()
    # Center
    centered_vertices = vertices - np.mean(vertices, axis=0)
    # Limit vertices to [-0.95, 0.95]
    max_abs = np.max(np.abs(centered_vertices))
    vertices = centered_vertices / (max_abs / 0.95)  
    
    # Sort by Z, Y,X where Z is vertical    
    def sort_vertices(vertex):
        return vertex[2], vertex[1], vertex[0]   
 
    seen = OrderedDict()
    for point in vertices: 
      key = tuple(point)
      if key not in seen:
        seen[key] = point
        
    unique_vertices =  list(seen.values()) 
    sorted_vertices = sorted(unique_vertices, key=sort_vertices)
      
    vertices_as_tuples = [tuple(v) for v in vertices]
    sorted_vertices_as_tuples = [tuple(v) for v in sorted_vertices]

    vertex_map = {old_index: new_index for old_index, vertex_tuple in enumerate(vertices_as_tuples) for new_index, sorted_vertex_tuple in enumerate(sorted_vertices_as_tuples) if vertex_tuple == sorted_vertex_tuple} 
    reindexed_faces = [[vertex_map[face[0]], vertex_map[face[1]], vertex_map[face[2]]] for face in faces]   
    sorted_faces = [sorted(sub_arr) for sub_arr in reindexed_faces]  
    
    return np.array(sorted_vertices), np.array(sorted_faces)
 
 

def augment_mesh(vertices, scale_factor):     
    jitter_factor=0.01 
    possible_values = np.arange(-jitter_factor, jitter_factor , 0.0005) 
    offsets = np.random.choice(possible_values, size=vertices.shape) 
    vertices = vertices + offsets   
    
    vertices = vertices * scale_factor 
    return vertices


#load_shapenet("./shapenet", "./shapenet_csv_files", 10, 10)   
#Find the csv files with the labels in the ShapeNetCore.v1.zip, download at  https://huggingface.co/datasets/ShapeNet/ShapeNetCore-archive  
def load_shapenet(directory, csv_files, per_category, variations ):
    obj_datas = []   
    chosen_models_count = {}   
    id_info = {} 
    print(f"per_category: {per_category} min {min}")
     
    for csv_filename in os.listdir(csv_files):
        if csv_filename.endswith(".csv"): 
            csv_path = os.path.join(csv_files, csv_filename) 
            with open(csv_path, newline='', encoding="utf-8") as csvfile:
                csv_reader = csv.DictReader(csvfile)
                for row in csv_reader: 
                    full_id = row['fullId'].replace("3dw.", "")
                    wn_lemmas = row['wnlemmas'].replace(",", " ")
                    id_info[full_id] = wn_lemmas 
    
    possible_values = np.arange(0.75, 1.0 , 0.005) 
    scale_factors = np.random.choice(possible_values, size=variations) 
    
    for category in os.listdir(directory): 
        category_path = os.path.join(directory, category)   
        if os.path.isdir(category_path) == False:
            continue 
        num_files_in_category = len(os.listdir(category_path))
        print(f"{category_path} got {num_files_in_category} files") 
        chosen_models_count[category] = 0  
        
        for filename in os.listdir(category_path):
            if filename.endswith((".obj", ".glb", ".off")):
                file_path = os.path.join(category_path, filename)
                
                if chosen_models_count[category] >= per_category:
                    break
                
                if os.path.getsize(file_path) >  20 * 1024: # 20 kb limit = less then 400-600 faces
                    continue
                    
                if filename[:-4] not in id_info:
                    print("Unable to find id info for ", filename)
                    continue 
                
                chosen_models_count[category] += 1
                vertices, faces = get_mesh(file_path)   
                textName =  id_info[filename[:-4]]  
                
                if len(faces) > 800: 
                    continue
                
                for scale_factor in scale_factors: 
                    aug_vertices = augment_mesh(vertices.copy(), scale_factor)  
                    obj_data = {"vertices": torch.tensor(aug_vertices.tolist(), dtype=torch.float).to("cuda"), "faces":  torch.tensor(faces.tolist(), dtype=torch.long).to("cuda"),"texts": textName } 
                    obj_datas.append(obj_data)
                    
    print("="*25)
    print("Chosen models count for each category:")
    for category, count in chosen_models_count.items():
        print(f"{category}: {count}")
    
    total_chosen_models = sum(chosen_models_count.values())
    print(f"Total number of chosen models: {total_chosen_models}")
    return obj_datas

  
   
def load_filename(directory, variations):
    obj_datas = []   
    chosen_models_count = {}      
    possible_values = np.arange(0.75, 1.0 , 0.005) 
    scale_factors = np.random.choice(possible_values, size=variations) 
    
    for filename in os.listdir(directory):
        if filename.endswith((".obj", ".glb", ".off")): 
            file_path = os.path.join(directory, filename) 
            vertices, faces = get_mesh(file_path)  
            
            faces = torch.tensor(faces.tolist(), dtype=torch.long).to("cuda")
            face_edges =  derive_face_edges_from_faces(faces)  
            texts, ext = os.path.splitext(filename)     
            
            for scale_factor in scale_factors: 
                aug_vertices = augment_mesh(vertices.copy(), scale_factor)  
                obj_data = {"vertices": torch.tensor(aug_vertices.tolist(), dtype=torch.float).to("cuda"), "faces":  faces, "face_edges" : face_edges, "texts": texts } 
                obj_datas.append(obj_data)
                     
    print(f"[create_mesh_dataset] Returning {len(obj_data)} meshes")
    return obj_datas

In [ ]:
from pathlib import Path 
import gc    
import torch
import os
from meshgpt_pytorch import MeshDataset

torch.cuda.empty_cache()
gc.collect()  

 
project_name = "demo_mesh" 

working_dir = f'.\{project_name}'

working_dir = Path(working_dir)
working_dir.mkdir(exist_ok = True, parents = True)
dataset_path = working_dir / (project_name + ".npz")


if not os.path.isfile(dataset_path):
    data = load_filename("./demo_mesh",50)  
    dataset = MeshDataset(data) 
    dataset.generate_face_edges() 
    print(set(item["texts"] for item in dataset.data)  ) 
    dataset.save(dataset_path)
 
dataset = MeshDataset.load(dataset_path) 
print(dataset.data[0].keys())
 

### Inspect

In [ ]:
from pathlib import Path
 
folder = working_dir / f'renders' 
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)
   
all_vertices = []
all_faces = []
vertex_offset = 0
translation_distance = 0.5  

for r, item in enumerate(data): 
    vertices_copy =  np.copy(item['vertices'])
    vertices_copy += translation_distance * (r / 0.2 - 1) 
    
    for vert in vertices_copy:
        vertex = vert.to('cpu')
        all_vertices.append(f"v {float(vertex[0])}  {float(vertex[1])}  {float(vertex[2])}\n") 
    for face in item['faces']:
        all_faces.append(f"f {face[0]+1+ vertex_offset} {face[1]+ 1+vertex_offset} {face[2]+ 1+vertex_offset}\n")  
    vertex_offset = len(all_vertices)
 
obj_file_content = "".join(all_vertices) + "".join(all_faces)
 
obj_file_path = f'{folder}/3d_models_inspect.obj'

with open(obj_file_path, "w") as file:
    file.write(obj_file_content)    
    

### Train!

In [ ]:
autoencoder = MeshAutoencoder().to("cuda")      

**Have at least 400-2000 items in the dataset, use this to multiply the dataset**  

In [ ]:
dataset.data = [dict(d) for d in dataset.data] * 10
print(len(dataset.data))

*Load previous saved model if you had to restart session*

In [ ]:
autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100, batch_size=8,  grad_accum_every=1, learning_rate = 1e-4) 
autoencoder_trainer.load(f'{working_dir}\mesh-encoder_{project_name}.pt')   
autencoder = autoencoder_trainer.model
for param in autoencoder.parameters():
    param.requires_grad = True

**Train to about 0.3 loss if you are using a small dataset**

In [ ]:

autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                             batch_size=8,
                                             grad_accum_every=2,
                                             learning_rate = 1e-2) 
loss = autoencoder_trainer.train(280,stop_at_loss = 0.7, diplay_graph= True)   
  
autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                             batch_size=8,
                                             grad_accum_every=2,
                                             learning_rate = 4e-3) 
loss = autoencoder_trainer.train(280,stop_at_loss = 0.28, diplay_graph= True)     

In [ ]:
autoencoder_trainer.save(f'{working_dir}\mesh-encoder_{project_name}.pt')   

In [ ]:
import gc  
torch.cuda.empty_cache()
gc.collect()  

max_length =  max(len(d["faces"]) for d in dataset if "faces" in d) 
max_seq = max_length * 6  
print("Highest face count:" , max_length)
print("Max token sequence:" , max_seq) 

transformer = MeshTransformer(
    autoencoder,
    dim = 512,
    coarse_pre_gateloop_depth = 6, # Better performance using more gateloop layers
    fine_pre_gateloop_depth= 4, 
    #attn_depth = 24, # GPT-2 medium have 24 layer depth, change if needed
    max_seq_len = max_seq, 
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", ## Change or remove this line if you are using:  https://github.com/MarcusLoppe/classifier-free-guidance-pytorch
    text_condition_cond_drop_prob = 0.0
) 

total_params = sum(p.numel() for p in transformer.decoder.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Decoder total parameters: {total_params}") 
total_params = sum(p.numel() for p in transformer.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Total parameters: {total_params}") 

## **Required!**, embed the text and run generate_codes to save 4-96 GB VRAM (dependant on dataset) ##

**If you don't;** <br>
During each during each training step the autoencoder will generate the codes and the text encoder will embed the text.
<br>
After these fields are generate: **they will be deleted and next time it generates the code again:**<br>

This is due to the dataloaders nature, it writes this information to a temporary COPY of the dataset


In [ ]:
labels = set(item["texts"] for item in dataset.data)
print(labels)
dataset.embed_texts(transformer)
dataset.generate_codes(autoencoder)
print(dataset.data[0].keys())

*Load previous saved model if you had to restart session*

In [ ]:
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=1,num_train_steps=100, dataset = dataset, learning_rate = 1e-1, batch_size=2)
trainer.load(f'{working_dir}\mesh-transformer_{project_name}.pt')  
transformer = trainer.model

**Train to about 0.0001 loss (or less) if you are using a small dataset**

In [ ]:
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=4,num_train_steps=100, dataset = dataset,
                                 learning_rate = 1e-3, batch_size=2)  
loss = trainer.train(100, stop_at_loss = 0.009) 

trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=4,num_train_steps=100, dataset = dataset,
                                 learning_rate = 5e-4, batch_size=2)
loss = trainer.train(200, stop_at_loss = 0.00001)  

In [ ]:
 
trainer.save(f'{working_dir}\mesh-transformer_{project_name}.pt')   

## Generate and view mesh

In [ ]:
def combind_mesh(path, mesh):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5  

    for r, faces_coordinates in enumerate(mesh): 
        numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)   
    
        for vertex in numpy_data:
            all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
    
        for i in range(1, len(numpy_data), 3):
            all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")
    
        vertex_offset += len(numpy_data)
    
    obj_file_content = "".join(all_vertices) + "".join(all_faces)
     
    with open(path , "w") as file:
        file.write(obj_file_content)   
 
def combind_mesh_with_rows(path, meshes):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5  
    
    for row, mesh in enumerate(meshes): 
        for r, faces_coordinates in enumerate(mesh): 
            numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)  
            numpy_data[:, 0] += translation_distance * (r / 0.2 - 1)  
            numpy_data[:, 2] += translation_distance * (row / 0.2 - 1)  
        
            for vertex in numpy_data:
                all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
            for i in range(1, len(numpy_data), 3):
                all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")
        
            vertex_offset += len(numpy_data)
        
        obj_file_content = "".join(all_vertices) + "".join(all_faces)
     
    with open(path , "w") as file:
        file.write(obj_file_content)   
        
        
def write_mesh_output(path, coords):
    numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)  
    obj_file_content = ""
    
    for vertex in numpy_data:
        obj_file_content += f"v {vertex[0]} {vertex[1]} {vertex[2]}\n"

    for i in range(1, len(numpy_data), 3):
        obj_file_content += f"f {i} {i + 1} {i + 2}\n"
 
    with open(path, "w") as file:
        file.write(obj_file_content) 
         

**Using only text**

In [ ]:
 
from pathlib import Path
 
folder = working_dir / 'renders'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)  
 
text_coords = [] 
for text in labels:
    print(f"Generating {text}")
    faces_coordinates = transformer.generate(texts = [text],  temperature = 0.0) 
    text_coords.append(faces_coordinates)
    
    write_mesh_output(f'{folder}/3d_output_{text}.obj', faces_coordinates)  
     
 
combind_mesh(f'{folder}/3d_models_all.obj', text_coords)

**Text + prompt of tokens**

Grab fresh copy of dataset

In [ ]:
dataset = MeshDataset.load(dataset_path)
dataset.generate_codes(autoencoder)

**Prompt with 10% of codes/tokens**

In [ ]:
from pathlib import Path
token_length_procent = 0.10 
codes = []
texts = []
for label in labels:
    for item in dataset.data: 
        if item['texts'] == label:
            num_tokens = int(item["codes"].shape[0] * token_length_procent) 
            
            texts.append(item['texts']) 
            codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))  
            break
        
folder = working_dir / f'renders/text+codes'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)  

coords = [] 



for text, prompt in zip(texts, codes): 
    print(f"Generating {text} with {prompt.shape[1]} tokens")
    faces_coordinates = transformer.generate(texts = [text],  prompt = prompt, temperature = 0) 
    coords.append(faces_coordinates) 
    
    obj_file_path = f'{folder}/{text}_{prompt.shape[1]}_tokens.obj'
    write_mesh_output(obj_file_path, faces_coordinates)
        
    print(obj_file_path)
     
 
combind_mesh(f'{folder}/text+prompt_all.obj', coords)

if text_coords is not None: 
    combind_mesh_with_rows(f'{folder}/both_verisons.obj', [text_coords , coords])

**Prompt with 0% to 80% of tokens**

In [ ]:
from pathlib import Path
 
folder = working_dir / f'renders/text+codes_rows'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)   

mesh_rows = []
for token_length_procent in np.arange(0, 0.8, 0.1):
    codes = []
    texts = []
    for label in labels:
        for item in dataset.data: 
            if item['texts'] == label:
                num_tokens = int(item["codes"].shape[0] * token_length_procent) 
                
                texts.append(item['texts']) 
                codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))  
                break
            
    coords = []   
    for text, prompt in zip(texts, codes): 
        
        print(f"Generating {text} with {prompt.shape[1]} tokens")
        faces_coordinates = transformer.generate(texts = [text],  prompt = prompt, temperature = 0) 
        coords.append(faces_coordinates)
        
        obj_file_path = f'{folder}/{text}_{prompt.shape[1]}_tokens.obj'
        write_mesh_output(obj_file_path, coords)  
        print(obj_file_path)
        
        
    mesh_rows.append(coords) 
    combind_mesh(f'{folder}/text+prompt_all_{token_length_procent}.obj', coords)
    
combind_mesh_with_rows(f'{folder}/all.obj', mesh_rows)
 

**Just some testing for text embedding similarity**

In [ ]:
import numpy as np 
texts = list(labels)
vectors = [transformer.conditioner.text_models[0].embed_text([text], return_text_encodings = False).cpu().flatten() for text in texts]
 
max_label_length = max(len(text) for text in texts)
 
# Print the table header
print(f"{'Text':<{max_label_length}} |", end=" ")
for text in texts:
    print(f"{text:<{max_label_length}} |", end=" ")
print()

# Print the similarity matrix as a table with fixed-length columns
for i in range(len(texts)):
    print(f"{texts[i]:<{max_label_length}} |", end=" ")
    for j in range(len(texts)):
        # Encode the texts and calculate cosine similarity manually
        vector_i = vectors[i]
        vector_j = vectors[j]
        
        dot_product = torch.sum(vector_i * vector_j)
        norm_vector1 = torch.norm(vector_i)
        norm_vector2 = torch.norm(vector_j)
        similarity_score = dot_product / (norm_vector1 * norm_vector2)
        
        # Print with fixed-length columns
        print(f"{similarity_score.item():<{max_label_length}.4f} |", end=" ")
    print()